# Основы программирования в Python

*Алла Тамбовцева, НИУ ВШЭ*

### Работа с API: пример с API ВКонтакте

Сегодня мы немного поработаем с API. API – программный интерфейс приложения, сокращение от *Application Programming Interface*. Этот интерфейс позволяет выполнять различные операции автоматически, через приложение. Если API нам нужен исключительно как источник данных, можно писать запросы, позволяющие обратиться к хранилищу информации внутри API. Если мы хотим управлять приложением, которое будет выполнять какие-то действия, удаленно, можно написать код, который будет, например, автоматически отвечать на сообщения, когда мы не онлайн, лайкать новый пост друга через 30 секунд после его появления, пересылать на почту фотографии, которые выложили участники диалога и прочее.

Мы будем работать с API социальной сети ВКонтакте. Использовать API для написания и приема сообщений средствами Python мы не будем, а рассмотрим API как источник данных, позволяющий выгрузить все посты со стены пользователя или сообщества. Для работы нам понадобится библиотека `vk`, ее нужно установить через `pip install vk` в *Anaconda Command Prompt* (см. пример с `selenium` [здесь](https://nbviewer.jupyter.org/github/allatambov/Py-programming-3/blob/master/11-06/lect-selenium1.ipynb)).

Импортируем библиотеку:

In [1]:
import vk

Теперь нужно авторизоваться: создать приложение и получить токен доступа к нему.

#### Для авторизации:
    
1. [Создать](https://vk.com/apps?act=manage) приложение ВКонтакте (пройти по ссылке). Дать название, выбрать *Standalone-приложение*.
2. Включить приложение, сделать публичным (*Настройки - Состояние* и выбрать в выпадающем меню *Приложение включено и видно всем*).
3. Авторизоваться: скопировать строку ниже в браузер, в `client id` вместо 1 выставить `id` своего приложения (первая строка в настроках ‒ *ID приложения*). Если не хочется ни в чем ограничивать свое приложение, можно оставить `scope=all` (у приложения будет доступ ко всему, к чему есть доступ у пользователя).

    > ```https://oauth.vk.com/authorize?client_id=1&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token```

4. Скопировать `access token` из обновленной адресной строки (все после `access_token=` и до `&expires_in`, без `&`). Никому не показывать! По этому токену можно получить доступ ко всему аккаунту.

In [2]:
# скопировать свой access token без пробелов вместо 1234
token = '1234'

In [3]:
session = vk.Session(access_token = token) # открыть сессию для работы
api = vk.API(session) # подключиться к API

Попробуем сгрузить первые 100 постов со [стены](https://vk.com/hseofficial) неофициального сообщества ВШЭ. Сохраним ссылку на сообщество в переменную `group`:

In [4]:
group = 'hseofficial'

Обратите внимание: ссылка должна быть относительной, то есть без части с `https:/vk.com/`. Python и так будет знать, что мы работаем с сетью ВКонтакте.

Теперь получим доступ к стене этого сообщества:

In [6]:
# wall.get - "подключаемся" к стене
# count - сколько постов выгрузить (максимум)
# v - версия API, можно обойтись без нее, но Python может выдать warning

res = api.wall.get(domain = group, count = 100, v = 5.103)

In [8]:
res

Результат, сохраненный в `res`, очень похож на словарь. На самом деле, многие API возвращают результаты в формате JSON (JavaScript Object Notation), который тоже может быть представлен в виде набора пар ключ-значение.

In [9]:
res.keys()

dict_keys(['count', 'items'])

Ключами являются `count` и `items`. Нужные нам объекты (текст постов, id автора, дата и время публикации и проч.) находятся в `items`.

In [10]:
res['items'][0] # первый элемент items - первый пост со всей информацией о нем

{'id': 33029,
 'from_id': -132,
 'owner_id': -132,
 'date': 1575971849,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': 'Что меня огорчает',
 'attachments': [{'type': 'link',
   'link': {'url': 'https://www.hse.ru/our/news/323346844.html',
    'title': 'Что меня огорчает',
    'caption': 'www.hse.ru',
    'description': 'Авторская колонка первого проректора Вадима Радаева',
    'photo': {'id': 457289885,
     'album_id': -26,
     'owner_id': 2000021743,
     'sizes': [{'type': 'k',
       'url': 'https://sun1-23.userapi.com/c858024/v858024349/120e4c/to-wJyYBh9w.jpg',
       'width': 1000,
       'height': 480},
      {'type': 'l',
       'url': 'https://sun1-86.userapi.com/c858024/v858024349/120e4b/jvKn7gctoWs.jpg',
       'width': 537,
       'height': 240},
      {'type': 'm',
       'url': 'https://sun1-90.userapi.com/c858024/v858024349/120e48/c9FrIN8g3F0.jpg',
       'width': 130,
       'height': 87},
      {'type': 'p',
       'url': 'https://sun1-17.userapi.com/c858024/v858

Помимо текста поста можно найти много всего интересного. Например, тип поста (`post_type`), дата (`date`), id поста (`id`), лайки (`likes`, которые включают информацию о том, могут ли пользователи лайкать пост и публиковать его, а также собственно число лайков), репосты (`reposts`, которые включают число репостов), число просмотров (`views`), комментарии (`comments`, которые включают информацию о том, могут ли пользователи комментировать пост, и число комментариев), и так далее.

Давайте остановимся на тексте поста, id автора, id поста, дате публикации, числе лайков и репостов. Чтобы извлечь соответствующую информацию, сохраним `items` и извлечем из них нужные поля:

In [11]:
items = res['items']

In [12]:
full_list = []

for item in items:
    L = [item['from_id'], item['id'], item['text'], item['date'],
        item['likes']['count'], item['reposts']['count']]  # нужные поля
    full_list.append(L)  # добавляем в список списков full_list

In [13]:
# несколько элементов списка
full_list[0:3]

[[-132, 33029, 'Что меня огорчает', 1575971849, 8, 0],
 [-132, 33028, '', 1575971665, 13, 2],
 [-132, 33027, '', 1575971625, 4, 0]]

Видно, что в некоторых постах текста не обнаружено. 

Из этого списка списков можно легко сделать датафрейм `pandas`. Но прежде посмотрим, как сгрузить следующие 100 (и не только 100) постов со стены. Обычно при работе с API нужно принимать во внимание две вещи: 1) какое ограничение стоит на число запросов за один раз (число постов в нашем случае); 2) какое ограничение стоит на число запросов в минуту. Чтобы действовать в соответствии с этими ограничениями, поступим так: будем грузить каждые следующие 100 постов, добавлять их к нашему списку, потом немного ждать, и грузить еще 100 постов, и так до тех пор, пока не сгрузим желаемое количество.

Давайте для начала выберем это желаемое число постов. Пусть будет 400. 

In [14]:
nposts = 400

Теперь вопрос: по каким значениям нужно «пробегаться» в цикле, чтобы сгрузить посты с 100 по 400 (первые 100 уже сохранены в `res`)? По целым значениям от 2 до 4 включительно, умножая эти значения на 100. В `vk.get` есть опция `offset`. Она позволяет сдвинуть начало отсчета постов на некоторое число. Так, если мы напишем `api.wall.get(domain = group, count = 100, offset = 100, v = 5.73)`,  мы получим посты с 100 по 200, так как начало отсчета сдвинулось на 100.

Реализуем описанное выше. Для цикла нам понадобится `range()`, а для задержки после выгрузки каждой сотни постов нам пригодится функция `sleep`: 

In [15]:
from time import sleep

In [16]:
for i in range(1, 3):
    res2 = api.wall.get(domain = group, count = 100, offset = 100 * i, v = 5.73)
    items2 = res2['items']
    items.extend(items2) # добавляем к первой сотне постов в items
    sleep(0.5)

In [17]:
# опять выберем только нужные поля
full_list = []
for item in items:
    L = [item['from_id'], item['id'], item['text'], item['date'],
        item['likes']['count'], item['reposts']['count']]
    full_list.append(L)

Оставлось превратить обновленный список `items` (список списков) в датафрейм. Импортируем `pandas`.

In [18]:
import pandas as pd

Создадим датафрейм:

In [19]:
df = pd.DataFrame(full_list)

In [20]:
df.head(10)

,0,1,2,3,4,5
0,-132,33029,Что меня огорчает,1575971849,8,0
1,-132,33028,,1575971665,13,2
2,-132,33027,,1575971625,4,0
3,-132,33026,,1575971586,8,1
4,-132,33025,"Вышка в прошлом году запустила новый проект, о...",1575971433,8,0
5,-132,33024,"Поздравляем! Основоположник ""Русских хакеров"" ...",1575971123,6,0
6,-132,33023,Опубликованы решения Ученого совета по стоимос...,1575970990,1,0
7,-132,33022,,1575969388,4,0
8,-132,33021,,1575878182,3,0
9,-132,33019,Вакансия для наших студентов от Леруа Мерлен h...,1575878062,4,0


Ура! Осталось только дать внятные названия столбцам и разобраться, почему дата представлена в таком виде. что делать со столбцами, мы уже знаем.

In [21]:
df.columns = ['from_id', 'id', 'text', 'date', 'likes', 'reposts']
df.head(10)

,from_id,id,text,date,likes,reposts
0,-132,33029,Что меня огорчает,1575971849,8,0
1,-132,33028,,1575971665,13,2
2,-132,33027,,1575971625,4,0
3,-132,33026,,1575971586,8,1
4,-132,33025,"Вышка в прошлом году запустила новый проект, о...",1575971433,8,0
5,-132,33024,"Поздравляем! Основоположник ""Русских хакеров"" ...",1575971123,6,0
6,-132,33023,Опубликованы решения Ученого совета по стоимос...,1575970990,1,0
7,-132,33022,,1575969388,4,0
8,-132,33021,,1575878182,3,0
9,-132,33019,Вакансия для наших студентов от Леруа Мерлен h...,1575878062,4,0


С датой все интереснее. То, что указано в столбце `date`, это дата в виде UNIX-времени (POSIX-времени). Это число секунд, прошедших с 1 января 1970 года. Несмотря на то, что такой формат даты-времени кажется необычным, он довольно широко распространен в разных системах и приложениях (см. подробнее [здесь](https://ru.wikipedia.org/wiki/Unix-%D0%B2%D1%80%D0%B5%D0%BC%D1%8F)). Этот факт, конечно, радует, но хочется получить дату в более человеческом формате. Давайте напишем функцию для перевода UNIX-времени в формат *год-месяц-день-часы-минуты-секунды*. Для этого нам понадобится модуль *datetime*.

In [22]:
from datetime import datetime

In [23]:
def date_norm(date):
    d = datetime.fromtimestamp(date) # timestamp - UNIX-время в виде строки
    str_d = d.strftime("%Y-%m-%d %H:%M:%S") # %Y-%m-%d %H:%M:%S - год-месяц-день, часы:мин:сек
    return str_d

Применим нашу функцию к элементам столбца *date* и создадим новый – *date_norm*.

In [24]:
df['date_norm'] = df.date.apply(date_norm)

In [25]:
df.head()

,from_id,id,text,date,likes,reposts,date_norm
0,-132,33029,Что меня огорчает,1575971849,8,0,2019-12-10 12:57:29
1,-132,33028,,1575971665,13,2,2019-12-10 12:54:25
2,-132,33027,,1575971625,4,0,2019-12-10 12:53:45
3,-132,33026,,1575971586,8,1,2019-12-10 12:53:06
4,-132,33025,"Вышка в прошлом году запустила новый проект, о...",1575971433,8,0,2019-12-10 12:50:33


Можно разбить на части дату и время, а не сохранять одной строкой. Для этого нужно применить метод `.split()` для строк и «раздвинуть» столбцы – после разбиения получить не один столбец, а сразу несколько. 

In [27]:
df['date_norm'].str.split() # результат разбиения – набор списков

0      [2019-12-10, 12:57:29]
1      [2019-12-10, 12:54:25]
2      [2019-12-10, 12:53:45]
3      [2019-12-10, 12:53:06]
4      [2019-12-10, 12:50:33]
                ...          
295    [2018-01-23, 14:43:19]
296    [2018-01-17, 08:50:56]
297    [2018-01-10, 11:32:29]
298    [2018-01-10, 11:24:29]
299    [2018-01-05, 15:13:01]
Name: date_norm, Length: 300, dtype: object

In [28]:
df['date_norm'].str.split(expand = True) # теперь расширили – 2 столбца 

,0,1
0,2019-12-10,12:57:29
1,2019-12-10,12:54:25
2,2019-12-10,12:53:45
3,2019-12-10,12:53:06
4,2019-12-10,12:50:33
...,...,...
295,2018-01-23,14:43:19
296,2018-01-17,08:50:56
297,2018-01-10,11:32:29
298,2018-01-10,11:24:29


In [29]:
new_dat = df['date_norm'].str.split(expand = True) # сохраняем
df['Date'] = new_dat[0] # первый столбец – в Date
df['Time'] = new_dat[1]  # второй столбец – в Time

In [30]:
df.head()

,from_id,id,text,date,likes,reposts,date_norm,Date,Time
0,-132,33029,Что меня огорчает,1575971849,8,0,2019-12-10 12:57:29,2019-12-10,12:57:29
1,-132,33028,,1575971665,13,2,2019-12-10 12:54:25,2019-12-10,12:54:25
2,-132,33027,,1575971625,4,0,2019-12-10 12:53:45,2019-12-10,12:53:45
3,-132,33026,,1575971586,8,1,2019-12-10 12:53:06,2019-12-10,12:53:06
4,-132,33025,"Вышка в прошлом году запустила новый проект, о...",1575971433,8,0,2019-12-10 12:50:33,2019-12-10,12:50:33


Всё! Материалы о разных методах и функциях для `vk.api` можно найти в [официальной документации](https://vk.com/dev/manuals). 